In [1]:
from __future__ import print_function
import math

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt

import numpy as np
import pandas as pd
from sklearn import metrics
import seaborn as sns
import tensorflow as tf

from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
# from tensorflow.python.data import Dataset

tf.logging.set_verbosity(tf.logging.ERROR)

# read csv
train_data = pd.read_csv("./dataSet/train.csv")
test_data = pd.read_csv("./dataSet/test.csv")

test_data_copy = test_data.copy()
train_data_copy = train_data.copy()


In [2]:
# display.display(train_data.describe())
# display.display(train_data.info())

In [3]:
def construct_features(input_data):
    return (set([tf.feature_column.numeric_column(my_feature)
                 for my_feature in input_data]))

In [4]:
def train(train_example, learning_rate, batch_size, steps, periods, mod_dir):
    # Use sample method to seperate train dataset and validation dataset
    train_set = train_example.sample(frac=0.8, replace=False, random_state=100)
    cv_set = train_example.loc[set(train_example.index) - set(train_set.index)]
    feature_columns = construct_features(train_example.drop('SalePrice', axis=1))
    
    # Create train and cv input function
    train_input = tf.estimator.inputs.pandas_input_fn(
        x = train_set.drop('SalePrice', axis=1),
        y = train_set.SalePrice,
        num_epochs=None,
        shuffle=True,
        batch_size=batch_size
    )
    
    train_eval_input = tf.estimator.inputs.pandas_input_fn(
        x = train_set.drop('SalePrice', axis=1),
        y = train_set.SalePrice,
        num_epochs=1,
        shuffle=True,
    )
    
    cv_input = tf.estimator.inputs.pandas_input_fn(
        x = cv_set.drop('SalePrice', axis=1),
        y = cv_set.SalePrice,
        num_epochs=1,
        shuffle=False,
    )
    
    # Declare optimizer for estimator
    my_optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 2.5)
    estimator = tf.estimator.DNNRegressor(
        hidden_units=[256, 128, 64],
        feature_columns=feature_columns,
        optimizer=my_optimizer,
        model_dir=mod_dir
    )
    
    training_rmse = []
    validation_rmse = []
    
    for i in range (periods):
        # Train model
        print('%d period:' % (i + 1), end='')
        estimator.train(input_fn=train_input, steps=steps)
        
        # Evaluate model with validation dataset
        eval_cv = estimator.evaluate(input_fn=cv_input)
        print(eval_cv['average_loss'], end=' ')
        
        # Take a break and compute predictions.
        training_predictions = estimator.predict(input_fn=train_eval_input)
        training_predictions = np.array([item['predictions'][0] for item in training_predictions])
        validation_predictions = estimator.predict(input_fn=cv_input)
        validation_predictions = np.array([item['predictions'][0] for item in validation_predictions])
    
        # Compute training and validation loss.
        training_root_mean_squared_error = math.sqrt(
            metrics.mean_squared_error(training_predictions, train_set.SalePrice))
        validation_root_mean_squared_error = math.sqrt(
            metrics.mean_squared_error(validation_predictions, cv_set.SalePrice))        
        print("  RMSE : %0.2f, %0.2f" % (training_root_mean_squared_error, validation_root_mean_squared_error))
        
        # Add the loss metrics from this period to our list.
        training_rmse.append(training_root_mean_squared_error)
        validation_rmse.append(validation_root_mean_squared_error)
    print("Model training finished.")

  
    # Output a graph of loss metrics over periods.
    plt.ylabel("RMSE")
    plt.xlabel("Periods")
    plt.title("Root Mean Squared Error vs. Periods")
    plt.tight_layout()
    plt.plot(training_rmse, label="training")
    plt.plot(validation_rmse, label="validation")
    plt.legend()
    
    return estimator
        

In [5]:
def my_stupid_one_hot(data, feature, one_hot_map, feature_map):
    for i in range(len(feature_map)):
        data[feature_map[i]] = data[feature] == one_hot_map[i]
    return data

feature_map = ('ExterQual1', 'ExterQual2', 'ExterQual3', 'ExterQual4')
one_hot_map = ('Gd', 'TA', 'Ex', 'Fa')

train_data = my_stupid_one_hot(train_data, 'ExterQual', one_hot_map, feature_map)
test_data = my_stupid_one_hot(test_data, 'ExterQual', one_hot_map, feature_map)
# display.display(train_data.head(10))

feature_map = ('BsmtQual1', 'BsmtQual2', 'BsmtQual3', 'BsmtQual4')
one_hot_map = ('Gd', 'TA', 'Ex', 'Fa')

train_data = my_stupid_one_hot(train_data, 'BsmtQual', one_hot_map, feature_map)
test_data = my_stupid_one_hot(test_data, 'BsmtQual', one_hot_map, feature_map)


In [6]:
def normalize(series):
    return (series - series.mean()) / series.std()


In [10]:
# outlier and missing data
outlier_data_set = pd.DataFrame({
    '1stFlrSF':normalize(train_data['1stFlrSF'].apply(lambda x : min(x, 2500))),
    'GrLivArea':normalize(train_data['GrLivArea'].apply(lambda x : min(x, 3000))),
    'OverallQual':normalize(train_data['OverallQual']),
    'FullBath':normalize(train_data['FullBath']),
    'TotalBsmtSF':normalize(train_data['TotalBsmtSF'].apply(lambda x : min(x, 3000))),
    'BsmtQual1':train_data['BsmtQual1'],
    'BsmtQual2':train_data['BsmtQual2'],
    'BsmtQual3':train_data['BsmtQual3'],
    'BsmtQual4':train_data['BsmtQual4'],
    'SalePrice':np.log(train_data['SalePrice']),
})

estimator = train(
    learning_rate = 0.01,
    batch_size = 10,
    steps = 100,
    periods = 15,
    train_example = outlier_data_set,
    mod_dir='./tensorboard/outlier2/train1'
)

1 period:3.4669158   RMSE : 2.01, 1.86
2 period:0.5911114   RMSE : 0.94, 0.77
3 period:0.25810698   RMSE : 0.67, 0.51
4 period:0.16525164   RMSE : 0.63, 0.41
5 period:0.10309315   RMSE : 0.60, 0.32
6 period:0.08591546   RMSE : 0.57, 0.29
7 period:0.08968188   RMSE : 0.59, 0.30
8 period:0.07279361   RMSE : 0.61, 0.27
9 period:0.073611364   RMSE : 0.55, 0.27
10 period:0.040748052   RMSE : 0.56, 0.20
11 period:0.07107246   RMSE : 0.53, 0.27
12 period:0.075469345   RMSE : 0.64, 0.27
13 period:0.059171926   RMSE : 0.64, 0.24
14 period:0.055353537   RMSE : 0.49, 0.24
15 period:0.05405401   RMSE : 0.56, 0.23
Model training finished.


In [12]:
# Final prediction
def test_input(test_example):
    test_input_fn = tf.estimator.inputs.pandas_input_fn(
          x=test_example,
          num_epochs=1, # only to predict
          shuffle=False,
    )
    return test_input_fn

tmp = test_data['TotalBsmtSF'].copy().dropna()
tmp = test_data['TotalBsmtSF'].sum() / len(test_data['TotalBsmtSF'])
test_data['TotalBsmtSF'] = test_data['TotalBsmtSF'].fillna(tmp)
# print(test_simple_example['TotalBsmtSF'].isnull().sum())

tmp = test_data['GarageArea'].copy().dropna()
tmp = test_data['GarageArea'].sum() / len(test_data['GarageArea'])
test_data['GarageArea'] = test_data['GarageArea'].fillna(tmp)

test = pd.DataFrame({
    '1stFlrSF':normalize(test_data['1stFlrSF'].apply(lambda x : min(x, 2500))),
    'GrLivArea':normalize(test_data['GrLivArea'].apply(lambda x : min(x, 2500))),
    'OverallQual':normalize(test_data['OverallQual']),
    'FullBath':normalize(test_data['FullBath']),
    'TotalBsmtSF':normalize(test_data['TotalBsmtSF'].apply(lambda x : min(x, 2500))),
    'BsmtQual1':test_data['BsmtQual1'],
    'BsmtQual2':test_data['BsmtQual2'],
    'BsmtQual3':test_data['BsmtQual3'],
    'BsmtQual4':test_data['BsmtQual4'],
})

# display.display(test_data.info())
final_input = test_input(test)

predictions = list(estimator.predict(input_fn=final_input))
predicted_classes = [math.exp(float(prediction['predictions'][0])) for prediction in predictions]

evaluation = test_data_copy['Id'].copy().to_frame()
evaluation["SalePrice"] = predicted_classes
evaluation.to_csv("evaluation_submission.csv", index=False)
# evaluation["actual"] = train_data.SalePrice
display.display(evaluation.describe())
display.display(evaluation)


Id      SalePrice
count  1459.000000    1459.000000
mean   2190.000000  201306.788460
std     421.321334   89968.004677
min    1461.000000   13904.655360
25%    1825.500000  143374.680933
50%    2190.000000  180260.371456
75%    2554.500000  233322.999823
max    2919.000000  969744.711397

Id      SalePrice
0     1461  131728.706408
1     1462  186130.965065
2     1463  179561.198933
3     1464  192636.820372
4     1465  207408.778315
5     1466  205697.976402
6     1467  179498.363784
7     1468  183154.595840
8     1469  204360.353243
9     1470  118262.855635
10    1471  215222.062093
11    1472  100009.887006
12    1473  108611.586554
13    1474  169900.098766
14    1475  141835.326264
15    1476  422718.626617
16    1477  257057.259374
17    1478  330248.471337
18    1479  286273.244486
19    1480  420094.729720
20    1481  368993.700505
21    1482  231647.705164
22    1483  194727.900757
23    1484  174456.105385
24    1485  179910.531340
25    1486  217825.496579
26    1487  367748.322019
27    1488  264626.691718
28    1489  191347.814295
29    1490  200242.412424
...    ...            ...
1429  2890   92776.283426
1430  2891  198820.206790
1431  2892   57023.274632
1432  2893  171755.017691
1433  2894  150530.896941
1434  2895  320026.442774
1435  2896  293585.123443
1436  2897  211963.770815
1437  2898  213285.451602
1438  2899  269965.935050
1439  2900  158505.277316
1440  2901  202643.966037
1441  2902  194698.189904
1442  2903  334776.713865
1443  2904  363721.905707
1444  2905  202619.810434
1445  2906  216959.929013
1446  2907  153726.336962
1447  2908  128655.372590
1448  2909  188117.513937
1449  2910   90530.633613
1450  2911  103386.468042
1451  2912  171525.362282
1452  2913  103386.468042
1453  2914  103386.468042
1454  2915  103386.468042
1455  2916  103386.468042
1456  2917  166697.851297
1457  2918  141590.168202
1458  2919  261692.398427

[1459 rows x 2 columns]